In [1]:
# training_rcnn.ipynb
import sys
from pathlib import Path
import yaml
import torch
from torch.utils.data import DataLoader
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from utils import get_transform, collate_fn
from traffic_dataset import TrafficDataset
from trainer import train_and_validate, prepare_save_dir

c:\Users\Dawid\Desktop\traffic-detection\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
project_root = Path().resolve()
sys.path.append(str(project_root))

def load_config(path):
    with open(path, 'r') as file:
        return yaml.safe_load(file)

cfg = load_config(project_root / 'cfg' / 'rcnn.yaml')
data_cfg = load_config(project_root / 'dataset' / 'dataset_day.yaml')

In [ ]:
train_dataset_day = TrafficDataset(
    csv_file='dataset/annotations_train_balanced_day.csv',
    root_dir=Path("dataset/day/train"),
    transforms=get_transform(train=True, cfg=cfg)
)
val_dataset_day = TrafficDataset(
    csv_file='dataset/annotations_test_balanced_day.csv',
    root_dir=Path("dataset/day/test"),
    transforms=get_transform(train=False, cfg=cfg)
)

train_loader_day = DataLoader(train_dataset_day, batch_size=cfg['batch_size'], shuffle=True, collate_fn=collate_fn, num_workers=8)
val_loader_day = DataLoader(val_dataset_day, batch_size=cfg['batch_size'], shuffle=False, collate_fn=collate_fn, num_workers=8)

In [ ]:
save_dir_day = prepare_save_dir(project_root / "runs_rcnn" / "rcnn_day", cfg)

model_day = fasterrcnn_resnet50_fpn(pretrained=True)
in_features = model_day.roi_heads.box_predictor.cls_score.in_features
model_day.roi_heads.box_predictor = FastRCNNPredictor(in_features, data_cfg['nc'])

c:\Users\Dawid\Desktop\traffic-detection\venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Dawid\Desktop\traffic-detection\venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
print(torch.cuda.is_available())
print(torch.version.cuda)
torch.cuda.empty_cache()

True
12.1


In [ ]:
train_and_validate(model_day, cfg, data_cfg, train_loader_day, val_loader_day, save_dir_day)

[Epoch 1/50] Training:   0%|          | 0/1132 [00:16<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
train_dataset_night = TrafficDataset(
    csv_file='dataset/annotations_train_balanced_night.csv',
    root_dir=Path("dataset/night/train"),
    transforms=get_transform(train=True, cfg=cfg)
)
val_dataset_night = TrafficDataset(
    csv_file='dataset/annotations_test_balanced_night.csv',
    root_dir=Path("dataset/night/test"),
    transforms=get_transform(train=False, cfg=cfg)
)

train_loader_night = DataLoader(train_dataset_night, batch_size=cfg['batch_size'], shuffle=True, collate_fn=collate_fn, num_workers=8)
val_loader_night = DataLoader(val_dataset_night, batch_size=cfg['batch_size'], shuffle=False, collate_fn=collate_fn, num_workers=8)

In [ ]:
save_dir_night = prepare_save_dir(project_root / "runs_rcnn" / "rcnn_night", cfg)

model_night = fasterrcnn_resnet50_fpn(pretrained=True)
in_features = model_night.roi_heads.box_predictor.cls_score.in_features
model_day.roi_heads.box_predictor = FastRCNNPredictor(in_features, data_cfg['nc'])

In [ ]:
train_and_validate(model_day, cfg, data_cfg, train_loader_night, val_loader_night, save_dir_night)